# Software Requirements Specification for Chemistry Code
## Samuel J. Crawford




# Table of Contents
<a id="Sec:ToC"></a>

An outline of all sections included in this SRS is recorded here for easy reference.

- [Table of Contents](#Sec:ToC)
- [Reference Material](#Sec:RefMat)
 - [Table of Symbols](#Sec:ToS)
 - [Abbreviations and Acronyms](#Sec:TAbbAcc)
- [Introduction](#Sec:Intro)
 - [Scope of Requirements](#Sec:ReqsScope)
- [Requirements](#Sec:Requirements)
 - [Functional Requirements](#Sec:FRs)
 - [Non-Functional Requirements](#Sec:NFRs)
- [References](#Sec:References)

# Reference Material
<a id="Sec:RefMat"></a>

This section records information for easy reference.

## Table of Symbols
<a id="Sec:ToS"></a>

The symbols used in this document are summarized in the [Table of Symbols](#Table:ToS) along with their units. The symbols are listed in alphabetical order.

<a id="Table:ToS"></a>

|Symbol|Description|Units|
|:--- |:--- |:--- |
|$r$|Representation of a chemical equation|--|


## Abbreviations and Acronyms
<a id="Sec:TAbbAcc"></a>

<a id="Table:TAbbAcc"></a>

|Abbreviation|Full Form|
|:--- |:--- |
|ChemCode|Chemistry Code|


# Introduction
<a id="Sec:Intro"></a>

Chemical equations are common ways of representing chemical reactions but they must be balanced <a href=#lund2023>lund2023</a> . This process of balancing a chemical equation involves introducing coefficients before each chemical formula such that there are the same number of atoms of each element on the reactant and product sides of the chemical equation. Because balancing must be done before a given chemical reaction can be used <a href=#lund2023>lund2023</a> , it is useful to have a tool to automatically do this. This would improve the productivity of scientists and engineers and reduce the potential for human error. This program should balance a given chemical equation if it is feasible and if it is not, it should provide a descriptive message communicating this to the user. The program that performs these tasks as documented here will be called Chemistry Code (ChemCode).

The following section provides an overview of the Software Requirements Specification (SRS) for ChemCode. This section explains the purpose of this document, the scope of the requirements, the characteristics of the intended reader, and the organization of the document.

## Scope of Requirements
<a id="Sec:ReqsScope"></a>

The scope of the requirements includes all chemical equations with at most one more compound than element. Furthermore, it also includes all inputted chemical formulas that describe real chemical compounds, are formatted following a set of conventions, and only consist of atomic symbols and subscripts.

# Requirements
<a id="Sec:Requirements"></a>

This section provides the functional requirements, the tasks and behaviours that the software is expected to complete, and the non-functional requirements, the qualities that the software is expected to exhibit.

## Functional Requirements
<a id="Sec:FRs"></a>

This section provides the functional requirements, the tasks and behaviours that the software is expected to complete.

<div id=\"inputValues\">
Input-Values: Input the values from [Tab:ReqInputs](#Table:ReqInputs).

</div>
<div id=\"convertMatrix\">
Convert-to-Matrix: Convert the inputted chemical equation to matrix form.

</div>

<a id="Table:ReqInputs"></a>

|Symbol|Description|Units|
|:--- |:--- |:--- |
|$r$|Representation of a chemical equation|--|


## Non-Functional Requirements
<a id="Sec:NFRs"></a>

This section provides the non-functional requirements, the qualities that the software is expected to exhibit.

<div id=\"accurate\">
Accurate: Chemical equations are only useful if they are balanced, so computed coefficients should be exact.

</div>
<div id=\"verifiable\">
Verifiable: The code is tested with complete verification and validation plan.

</div>
<div id=\"reusable\">
Reusable: The code is modularized.

</div>
<div id=\"portable\">
Portable: The code is able to be run in different environments.

</div>

# References
<a id="Sec:References"></a>
<a id="lund2023"></a>
[1]: Lund, Lance. <em>Introduction to Chemistry</em>. Cambridge and Coon Rapids, MN, USA: LibreTexts, January, 2023. <a href="https://chem.libretexts.org/Courses/Anoka-Ramsey_Community_College/Introduction_to_Chemistry">https://chem.libretexts.org/Courses/Anoka-Ramsey_Community_College/Introduction_to_Chemistry</a>. Print.

